# Analyzer Evaluation Demonstration Notebook

This notebook demonstrates how to use the functions in `source_analyzer_evaluation.py` to evaluate three analyzers:

- **AISAnalyzer**: Run on a groundtruth dataset called `vessel_groundtruth`.
- **InfrastructureAnalyzer**: Run on a dataset called `infrastructure_groundtruth`.
- **DarkAnalyzer**: Run on a dataset called `dark_vessel_groundtruth`.

For AISAnalyzer and InfrastructureAnalyzer, we label results using the source name (`st_name`). For DarkAnalyzer, we label results based on the spatial distance (using a 0.005° threshold) to the ground truth dark vessel location.


In [ ]:
import os
import sys
from dotenv import load_dotenv

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from IPython.display import clear_output

load_dotenv()

# Set the download path for demonstration and ensure the folder exists.
download_path = os.getenv("ASA_DOWNLOAD_PATH")
os.makedirs(download_path, exist_ok=True)

git_path = os.getenv("GIT_FOLDER")
cv3_path = os.getenv("CV3_FOLDER") 
sys.path.append(git_path)
sys.path.append(cv3_path)

In [ ]:
from cerulean_cloud.cloud_function_asa.utils.analyzer import (
        AISAnalyzer,
        InfrastructureAnalyzer,
        DarkAnalyzer
)

In [ ]:
# Import functions from your script.
from asa_analysis.evaluation.source_analyzer_evaluation import (
    label_dark_vessel_results_with_distance,
    label_results_with_st_name,
    apply_labeling,
    calculate_metrics,
    plot_metrics,
    process_groundtruth_on_analyzer
)

## Load Groundtruth Datasets

Load the dark vessel dataset and the SAR detections. Then, load the CSV that contains hitl verification for vessels and infrastructure.

In [ ]:
dark_vess_df = pd.read_csv(r'refined_dark_vessel_dataset.csv')
sar_detections = pd.read_csv(r'sar_detections_hitl_dark_ds.csv')

# Keep only rows where 'mmsi' is NaN (i.e. true dark vessel detections)
# true_dark_vess_df = dark_vess_df[dark_vess_df['mmsi'].isna()]

# Toggle long distance coincidence: remove vessels ~20km or more away from slick.
long_distance_coincidence = [3581643, 3581482, 3581103, 3581287, 3581532, 3581538,
                             3582446, 3581900, 3582053, 3580996, 3581711, 3581920,
                             3581075, 3581141, 3581812, 3582235, 3582465, 3582774, 3582584]
dark_vess_df = dark_vess_df[[(slick_id not in long_distance_coincidence)
                              for slick_id in dark_vess_df['slick_id'].values]]

# Create a GeoDataFrame for dark vessels.
dark_vessel_groundtruth = gpd.GeoDataFrame(
    dark_vess_df, 
    geometry=gpd.points_from_xy(dark_vess_df['lon'], dark_vess_df['lat']),
    crs="EPSG:4326"
)

# Process SAR detections (for example purposes; not used further in this demo).
sar_detections_gdf = gpd.GeoDataFrame(
    sar_detections, 
    geometry=gpd.points_from_xy(sar_detections['detect_lon'], sar_detections['detect_lat']),
    crs="EPSG:4326"
)
sar_detections_gdf = sar_detections_gdf[sar_detections_gdf['structure_id'].isna()]
sar_detections_gdf = sar_detections_gdf.reset_index()

# Load hitl CSV for vessel and infrastructure groundtruth.
csv_path = r'slick_to_source dump 2024-12-31.csv'
hitl_df = pd.read_csv(csv_path)

# Extract groundtruth lists for infrastructure and vessels.
infrastructure_groundtruth = hitl_df[(hitl_df['type'] == 2) & (hitl_df['hitl_verification'])]
vessel_groundtruth = hitl_df[(hitl_df['type'] == 1) & (hitl_df['hitl_verification'])]

if "slick" in vessel_groundtruth.columns:
    vessel_groundtruth = vessel_groundtruth.rename(columns={"slick": "slick_id"})
if "slick" in infrastructure_groundtruth.columns:
    infrastructure_groundtruth = infrastructure_groundtruth.rename(columns={"slick": "slick_id"})

df = pd.read_csv(r'nonoise_SAR_Fixed_Infrastructure.csv')
gfw_gdf = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df['lon'], df['lat'])],
    crs="EPSG:4326"  # Set the coordinate reference system to WGS84
)



In [ ]:
vessel_groundtruth = vessel_groundtruth.iloc[[1,2,3,4,5,6,7,8,9,10]]
# infrastructure_groundtruth = infrastructure_groundtruth.iloc[[0]]
# dark_vessel_groundtruth = dark_vessel_groundtruth.iloc[[0]]

## Process Groundtruth with Each Analyzer

We use the `process_groundtruth_on_analyzer` function to run each analyzer over its respective groundtruth.
This function loops over each groundtruth row, downloads the associated slick and scene GeoJSON, and then computes
coincidence scores using the given analyzer class.


In [ ]:
# Process InfrastructureAnalyzer on infrastructure_groundtruth.
results_infra = process_groundtruth_on_analyzer(
    InfrastructureAnalyzer, infrastructure_groundtruth, points_gdf=gfw_gdf, analyzer_params={}
)

In [ ]:
# Process DarkAnalyzer on dark_vessel_groundtruth.
results_dark = process_groundtruth_on_analyzer(
    DarkAnalyzer, dark_vessel_groundtruth, points_gdf=sar_detections_gdf, analyzer_params={}
)

In [ ]:
results_vessel = process_groundtruth_on_analyzer(
    AISAnalyzer, vessel_groundtruth, analyzer_params={}
)

## Label the Results

For the AIS and Infrastructure analyzers, we label detections using the `st_name` by applying `label_results_with_st_name`. Both the results and the groundtruth dataframes must have `st_name` as the source identifier with type `int`.

For the DarkAnalyzer, we label based on distance using `label_dark_vessel_results_with_distance`.


In [ ]:
#postprocess outputs if necessary to align with groundtruth
results_infra['st_name']=results_infra['structure_id']
results_vessel['st_name'] = results_vessel['st_name'].astype(int)

In [ ]:
results_vessel_labeled = apply_labeling(results_vessel, vessel_groundtruth, label_results_with_st_name)
results_infra_labeled = apply_labeling(results_infra, infrastructure_groundtruth, label_results_with_st_name)
# Label dark vessel results based on distance.
results_dark_labeled = apply_labeling(results_dark, dark_vessel_groundtruth, label_dark_vessel_results_with_distance)

## Calculate and Display Metrics

We now compute evaluation metrics (e.g., top-1 and top-3 source rates, average coincidence scores, and the ratio of true to false coincidence scores)
for each method.

In [ ]:
all_results = {
    "AISAnalyzer": results_vessel_labeled,
    "InfrastructureAnalyzer": results_infra_labeled,
    "DarkAnalyzer": results_dark_labeled
}

metrics_df = calculate_metrics(all_results)
print("Evaluation Metrics:")
plot_metrics(metrics_df)